### XML-Datei laden und Dataframe erstellen

XML (Extensible Markup Language) ist ein Datenformat, das meist zur Annotation von unstrukturierten und semi-strukturierten Daten verwendet und mit XSLT oder XQuery weiterverarbeitet wird. Dieses Notebook zeigt, wie XML-Dateien mit Python verarbeitet eingelesen und weiterverarbeitet werden können. Es fokussiert auf die Extraktion von strukturierten Daten aus der XML-Datei und die Verarbeitung eines Dataframes. Es stellt eine alternative Methode zur Extraktion von strukturierten Daten mittels XSLT dar.

Eine umfassende Einführung in die Verarbeitung von XML-Daten mit Python finden Sie bei Gunter Vasold: https://github.com/gvasold/gdp/blob/main/05-texte/03-xml.ipynb


Zur Verarbeitung von XML stehen in der Standardbibliothek die XML-Bibliotheken **dom.sax**, **xml.dom.minidom**, **xml.dom.pulldom** und **xml.etreeElementTree** zur Verfügung. 
Wir verwenden die Zusatzbibliothek `lxml` (http://lxml.de/). `lxml` ist eine Bibliothek zum Parsen und Interagieren von XML-Daten. Im unterschied zu `xml.etreeElementTree` ist XPath (die Abfragesprache für XML-Daten) in `lxml`vollständig implementiert. 

In [1]:
from lxml import etree
import pandas as pd

Nach dem Import der notwendigen Module wird der Pfad zur XML-Datei angegeben und die Funktion `etree.parse` aus `lxml` verwendet, um die Datei in eine Baumstruktur zu laden, in der navigiert und Abfragen generiert werden können. 

In [2]:
tree = etree.parse('ger000244-gottsched-der-sterbende-cato.xml')

Um im Baum navigieren zu können, wird zunächst die Wurzel ermittelt:

In [3]:
root = tree.getroot()

In [4]:
root.tag

'{http://www.tei-c.org/ns/1.0}TEI'

Den Textinhalt von root ausgeben (das ist in diesem Fall nur ein Zeilenumbruch)

In [5]:
root.text

'\n  '

Attribute von root ausgeben

In [6]:
root.attrib

{'{http://www.w3.org/XML/1998/namespace}id': 'ger000244', '{http://www.w3.org/XML/1998/namespace}lang': 'ger'}

Direkte Kindelemente von root ausgeben

In [7]:
for child in root:
    print(child)

<Element {http://www.tei-c.org/ns/1.0}teiHeader at 0x257ff53d400>
<Element {http://www.tei-c.org/ns/1.0}standOff at 0x25797561000>
<Element {http://www.tei-c.org/ns/1.0}text at 0x25797562440>


Die XML-Datei verwendet Namensräume; dieser muss entsprechend angegeben werden, um die Elemente überhaupt finden zu können. 

In [11]:
nsmap = {'tei': 'http://www.tei-c.org/ns/1.0'}

Um in der XML-Struktur navigierten zu können, wird XPath verwendet. Der XPath `//tei:sp/tei:speaker` findet alle Sprecher:innen, die mit dem Element `<speaker>` ausgezeichnet wurden.   

In [12]:
for speech in root.findall('.//tei:sp/tei:speaker', namespaces=nsmap):
    print(speech.text)

ARSENE.
PHÖNICE.
ARSENE.
PHÖNICE.
ARSENE.
PHÖNICE.
ARSENE.
PHÖNICE.
ARSENE.
CATO.
ARSENE.
CATO.
ARSENE.
CATO.
ARSENE.
CATO.
ARSENE.
CATO
PHOKAS.
CATO.
PHOKAS.
CATO.
PHOKAS.
ARTABANUS.
CATO
ARTABANUS.
CATO.
PHOKAS.
CATO.
PHOKAS.
CATO.
CATO.
PHARNACES.
CATO.
PHARNACES.
CATO.
PHARNACES.
CATO.
PHARNACES.
CATO.
PHARNACES.
CATO.
PHARNACES.
CATO.
FELIX.
CATO.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
DOMITIUS.
PHOKAS.
DOMITIUS.
PHOKAS.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
DOMITIUS.
CATO.
ARSENE.
DOMITIUS.
ARSENE.
PHARNACES.
ARSENE.
PHARNACES
ARSENE.
PHARNACES.
ARSENE.
PHARNACES.
PORCIUS.
ARSENE.
PORCIUS.
PHARNACES.
PORCIUS.
PHARNACES.
PORCIUS.
PHARNACES.
PORCIUS.
PHARNACES.
PORCIUS.
PHARNACES.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
FELIX.
PHARNACES.
CÄSAR.
DOMITIUS.
CÄSAR.
DOMITIUS.
CÄSAR.
DOMITIUS.
CÄSAR.
ARSENE
CÄSAR.
ARSENE.
CÄSAR.
ARSENE.
CÄSAR.
A

Im nachfolgenden Bespiel werden zunächst über XPath alle Sprechakte (`<sp>`-Elemente) adressiert. Die .// Syntax sucht nach diesen Elementen auf jedem beliebigen Level im XML Baum. Danach iteriert der Code über jeden Sprechakt. 

Der Name der Sprecher:in wird aus dem `who`-Attribut von jedem `<sp>`-Element extrahiert. Die Dictionaries `speech_count`und `line_count` werden dazu verwendet, um die Anzahl der Sprachakte und der Zeilen zu zählen. Wenn der/die Sprecher:in bereits im Dictionary enthalten ist, wird die Zahl um 1 erhöht. Wenn nicht, wird der/die Sprecher:in mit einem initialen Zähler 1 hinzugefügt.

In [13]:
speech_acts = tree.findall('.//tei:sp', namespaces=nsmap)

# Initialisierte Dictionaries für Sprechakte und Zeilen
speech_count = {}
line_count = {}

for sp in speech_acts:
    speaker = sp.get('who')
    
    if speaker:
        speaker = speaker.lstrip('#')
        
        # Count the speech acts
        speech_count[speaker] = speech_count.get(speaker, 0) + 1
        
        # Count the lines within each speech act
        lines = sp.findall('.//tei:l', namespaces=nsmap)
        line_count[speaker] = line_count.get(speaker, 0) + len(lines)

Die Daten werden kombiniert und das Dataframe erstellt. Das DataFrame mit dem Namen `df_speech`wird aus den Daten generiert, die in den Dictionaries `speech_count`und `line_count`gesammelt wurden. 

Creating a DataFrame:

`pd.DataFrame({...})`ist der Konstruktor, um ein neues DataFrame zu generieren. Beim DataFrame handelt es sich um zweidimensionale tabelarische Daten. Das DataFrame wird aus Dictionaries erstellt, bei dem jedes key-value Paar mit einer Zeile korrespondiert.

In [ ]:
df_speech = pd.DataFrame({
    'Speaker': speech_count.keys(),
    'Number of Speech Acts': speech_count.values(),
    'Number of Lines': [line_count[speaker] for speaker in speech_count.keys()]
})

# Anzeige des Dataframes
print(df_speech)


In [ ]:
df_speech